In [ ]:
%%capture
!pip install langchain openai

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Prompt Pipelining Overview

Prompt pipelining is a powerful tool designed for those who seek a modular and efficient approach to prompt design.


## Here are some common cases where prompt pipelining can be useful:

 - Reusing prompt components like introductions, instructions, examples, etc. across multiple prompts. Prompt pipelining allows easily reusing these modular blocks.

 - Splitting up a long prompt into smaller logical chunks. This can make prompts more readable and maintainable.

 - Dynamically assembling prompts based on conditionals or other logic. You can build prompts on the fly from reusable parts.

 - Creating prompts in a loop by appending to an existing base prompt. Useful for things like few-shot learning prompts.

 - Mixing static text with templates containing variables. The static text can provide structure while the templates inject dynamic content.

 - Composing chat prompts from message templates and static messages. Each piece gets appended as a new message.

 - Building up prompts from user provided components for customization. Prompt pipelining allows prompts to be assembled from modular parts.

 - Any situation where you want to build up prompts in a reusable composable way, prompt pipelining provides a clean interface for that.

## String Prompt Pipelining

For string prompts, templates are interconnected in sequence.

You have the liberty to utilize either direct prompts or strings.

However, it's essential to note that the initial element in the sequence should always be a prompt.

In [ ]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("I'm heading to {destination}. ")
    + "Recommend a great {activity} spot!")

prompt


PromptTemplate(input_variables=['destination', 'activity'], template="I'm heading to {destination}. Recommend a great {activity} spot!")

In [ ]:
prompt = prompt + "\n\nAlso, any local delicacies I should try?"
prompt

PromptTemplate(input_variables=['destination', 'activity'], template="I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try?")

In [ ]:
prompt.format(destination="Punjab", activity="dining")

"I'm heading to Punjab. Recommend a great dining spot!\n\nAlso, any local delicacies I should try?"

### The key differences between prompt pipelining and multi-input prompts are:

 - Prompt pipelining composes multiple prompts/strings into a single final prompt. Multi-input prompts allow passing multiple inputs to a single prompt template.

 - With pipelining, each component can be formatted independently before joining. Multi-input just formats a single template.

 - Pipelining joins components into a linear sequence. Multi-input prompts are a single template handling multiple inputs.

 - Pipelining allows reuse of components like introductions, examples, etc. Multi-input is focused on handling multiple query inputs.

 - Pipelining output is a single prompt. Multi-input output is the completion based on multiple inputs.

 - Pipelining lets you build up prompts modularly. Multi-input is one template handling multiple inputs.

In summary, prompt pipelining composes multiple prompt components into one final prompt in a linear sequence. Multi-input prompts allow a single template to handle multiple input variables in parallel. Pipelining focuses on modular prompt building, while multi-input handles multiple query inputs.

# Use in a chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt)
print(chain.run(destination="Punjab", activity="dining"))

If you're heading to Punjab, one great dining spot you should try is Kesar Da Dhaba in Amritsar. It is a famous restaurant known for its authentic Punjabi cuisine and has been serving delicious food since 1916.

When it comes to local delicacies, Punjab offers a range of mouthwatering dishes. Here are a few must-try delicacies:

1. Makki di Roti and Sarson da Saag: This is a classic Punjabi dish made with cornbread (makki di roti) served with a spicy mustard greens curry (sarson da saag). It's a must-try during the winter season.

2. Butter Chicken: A popular dish worldwide, this creamy and flavorful chicken curry is a Punjabi specialty. It's best enjoyed with naan or rice.

3. Amritsari Fish: If you're a seafood lover, don't miss out on trying Amritsari fish. It's a deep-fried fish marinated in a flavorful blend of spices, perfect for satisfying your taste buds.

4. Chole Bhature: This is a popular Punjabi breakfast dish. Chole is spicy chickpea curry, and bhature is a deep-fried brea

In [ ]:
prompt = prompt +" How should I greet the locals in a friendly, informal colloquial manner?"
chain = LLMChain(llm=model, prompt=prompt)
print(chain.run(destination="Punjab", activity="dining"))

If you're heading to Punjab, I highly recommend trying out "Kesar Da Dhaba" in Amritsar. It is a famous dining spot known for its delicious Punjabi cuisine and traditional flavors.

When it comes to local delicacies, you must try the famous "Sarson Ka Saag" (a mustard greens dish) with "Makki Ki Roti" (a corn flour bread). Another must-try is "Chole Bhature" (spiced chickpeas with fried bread), which is a popular Punjabi dish.

To greet the locals in a friendly, informal colloquial manner, you can use the phrase "Sat Sri Akal." It is a common Punjabi greeting that means "God is the ultimate truth." It is widely used and is sure to make the locals feel warm and welcomed.


# Example usecase

In [ ]:
class TravelChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI()
        self.base_prompt = PromptTemplate.from_template(base_template)

    def append_to_prompt(self, additional_text):
        self.base_prompt += additional_text

    def run_chain(self, destination, activity):
        chain = LLMChain(llm=self.model, prompt=self.base_prompt)
        return chain.run(destination=destination, activity=activity)

# Usage
base_template = "I'm heading to {destination}. Recommend a great {activity} spot!"
chatbot = TravelChatbot(base_template)
# Basic prompt
chatbot.run_chain(destination="Punjab", activity="dining")

"If you're heading to Punjab, I would highly recommend trying out 'Kesar Da Dhaba' in Amritsar. It is a renowned restaurant that has been serving delicious Punjabi cuisine since 1916. Their specialty is the 'Dal Makhani' - a rich and creamy lentil dish, along with other authentic Punjabi delicacies like 'Butter Chicken', 'Amritsari Kulcha', and 'Lassi' (a traditional Punjabi drink). The rustic ambiance and traditional Punjabi hospitality make it a must-visit dining spot for an authentic culinary experience in Punjab."

In [ ]:
# Append more to the prompt and run again
chatbot.append_to_prompt("\n\nAlso, any local delicacies I should try?")
print(chatbot.run_chain(destination="Punjab", activity="dining"))

If you're heading to Punjab, one great dining spot to try is Kesar Da Dhaba in Amritsar. It is a famous and iconic restaurant known for its delicious Punjabi cuisine.

As for local delicacies, here are a few you should definitely try:

1. Amritsari Kulcha: This is a popular Punjabi dish, a type of stuffed bread usually served with chole (chickpea curry) and a side of pickles.

2. Makki di Roti and Sarson da Saag: This is a traditional Punjabi dish made with cornmeal flatbread (makki di roti) and a spicy mustard greens curry (sarson da saag). It's a must-try during the winter season.

3. Butter Chicken: A classic Punjabi dish, butter chicken is a rich and creamy tomato-based chicken curry. It is usually served with naan or rice.

4. Lassi: A refreshing Punjabi drink made with yogurt, water, and sometimes flavored with cardamom or mango. It is a perfect accompaniment to your meals.

5. Paneer Tikka: This is a popular vegetarian dish made with marinated and grilled cottage cheese, served 

In [ ]:
chatbot.append_to_prompt(" How should I greet the locals in a friendly, informal, jolly colloquial manner?")
chatbot.run_chain(destination="Punjab", activity="dining")

'If you\'re heading to Punjab, one great dining spot you should try is Kesar Da Dhaba in Amritsar. It is a popular restaurant known for its delicious Punjabi cuisine, especially their signature dish, Dal Makhani.\n\nWhen it comes to local delicacies, you must try Makki di Roti (cornbread) with Sarson da Saag (mustard greens curry), Butter Chicken, Chole Bhature (spiced chickpeas with fried bread), and Lassi (a refreshing yogurt-based drink). These are some of the famous Punjabi dishes that will give you a true taste of the region.\n\nTo greet the locals in a colloquial manner, you can say "Sat Sri Akal" which means "May God be with you." It\'s a common and respectful way to greet people in Punjab.\n\nTo greet the locals in a friendly, informal, and jolly colloquial manner, you can use "Ki haal hai?" which translates to "How are you?" and is widely used among friends and acquaintances. This will help you strike up a friendly conversation and make the locals feel more comfortable with yo

# Chat Prompt Pipeline

A chat prompt is made up a of a list of messages. Purely for developer experience, we've added a convinient way to create these prompts. In this pipeline, each new element is a new message in the final prompt.

### Key Insights into Chat Prompt Pipelining in Langchain:

Composition: Chat prompt pipelining facilitates the creation of chat prompts using reusable message components. Each addition to the pipeline translates to a new message in the final chat prompt.

Versatility: You can seamlessly integrate static message objects, such as HumanMessage and AIMessage, with variable-containing message templates.

End Result: The culmination is a unified ChatPromptTemplate, primed for formatting and integration into a chain. This structure promotes the effortless reuse of components, including instructions and examples.

Modularity: This approach champions the dynamic construction of chat prompts using modular blocks, ensuring flexibility and efficiency.

So in summary, chat prompt pipelining composes chat prompts from reusable message templates and static messages. It allows dynamically constructing prompts from logical blocks in a user friendly way. The end result is a single reusable ChatPromptTemplate.



In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Setting the scene with a Cockney-themed system message
prompt = SystemMessage(content="Welcome to the East End Cockney Chat! 🇬🇧")

# Constructing a chat flow with dry humour
new_prompt = (
    prompt
    + HumanMessage(content="Alright, guv'nor?")
    + AIMessage(content="Not too shabby. Did you hear about the London fog?")
    + "{input}"
)

# Formatting the chat with the user's response
new_prompt.format_messages(input="No, what about it?")
print(new_prompt)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

model = ChatOpenAI()

chain = LLMChain(llm=model, prompt=new_prompt)

# Running the chatbot to get the punchline
response = chain.run("No, what about it?")
response

input_variables=['input'] messages=[SystemMessage(content='Welcome to the East End Cockney Chat! 🇬🇧'), HumanMessage(content="Alright, guv'nor?"), AIMessage(content='Not too shabby. Did you hear about the London fog?'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


"Well, the fog's thicker than a proper pea soup these days! Can't see my own two feet, I tell ya. Makes it a right challenge to get around town."

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

class CockneyChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI()
        self.base_prompt = ChatPromptTemplate.from_template(base_template)

    def append_to_prompt(self, message_type, content):
        if message_type == "human":
            self.base_prompt += HumanMessage(content=content)
        elif message_type == "ai":
            self.base_prompt += AIMessage(content=content)
        else:
            raise ValueError("Invalid message type")

    def run_chain(self, input_message):
        chain = LLMChain(llm=self.model, prompt=self.base_prompt)
        return chain.run(input=input_message)

# Usage
base_template = "Welcome to the East End Cockney Chat! 🇬🇧"
chatbot = CockneyChatbot(base_template)

In [ ]:
chatbot.append_to_prompt("human", "Alright, guv'nor?")
chatbot.append_to_prompt("ai", "Not too shabby. Did you hear about the London fog?")
print(chatbot.run_chain(("human","No, what about it?")))

Aye, I 'eard about it. Thick as pea soup, it was. Couldn't see me own hand in front of me face!


In [ ]:
chatbot.append_to_prompt("ai", "It mist the bus this morning.")
print(chatbot.run_chain("That's a proper knees-up!"))

Aye, it's a proper pea-souper out there, innit? Can't see a bloomin' thing!


In [ ]:
chatbot.base_prompt

ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Welcome to the East End Cockney Chat! 🇬🇧')), HumanMessage(content="Alright, guv'nor?"), AIMessage(content='Not too shabby. Did you hear about the London fog?'), AIMessage(content='It mist the bus this morning.')])

# Prompt Composition

This can be useful when you want to reuse parts of prompts. This can be done with a PipelinePrompt. A PipelinePrompt consists of two main parts:

1. Final prompt: The final prompt that is returned

2. Pipeline prompts: A list of tuples, consisting of a string name and a prompt template. Each prompt template will be formatted and then passed to future prompt templates as a variable with the same name.

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [ ]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [ ]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [ ]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [ ]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [ ]:
pipeline_prompt.input_variables

['person', 'input', 'example_q', 'example_a']

In [ ]:
last_prompt = pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Tesla",
    input="What's your favorite social media site?"
)

print(last_prompt)

You are impersonating Elon Musk.

Here's an example of an interaction: 

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


In [ ]:
from langchain.llms import OpenAI

llm=OpenAI()

llm(last_prompt)

' Twitter!'

In [ ]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

class CookingShowChatbot:
    def __init__(self):
        # Base template for the cooking show scenario
        self.full_template = """{introduction}
        {example_dish}

        {present_dish}"""
        self.full_prompt = PromptTemplate.from_template(self.full_template)

        # Introduction where the user impersonates a famous chef
        self.introduction_template = """Welcome to the cooking show! Today, you're channeling the spirit of Chef {chef_name}."""
        self.introduction_prompt = PromptTemplate.from_template(self.introduction_template)

        # Example dish made by the famous chef
        self.example_dish_template = """Remember when Chef {chef_name} made that delicious {example_dish_name}? It was a hit!"""
        self.example_dish_prompt = PromptTemplate.from_template(self.example_dish_template)

        # User's turn to present their dish
        self.present_dish_template = """Now, it's your turn! Show us how you make your {user_dish_name}. Let's get cooking!"""
        self.present_dish_prompt = PromptTemplate.from_template(self.present_dish_template)

        # Combining the prompts into a pipeline
        self.input_prompts = [
            ("introduction", self.introduction_prompt),
            ("example_dish", self.example_dish_prompt),
            ("present_dish", self.present_dish_prompt)
        ]
        self.pipeline_prompt = PipelinePromptTemplate(final_prompt=self.full_prompt,
                                                      pipeline_prompts=self.input_prompts
                                                      )

    def run_scenario(self, chef_name, example_dish_name, user_dish_name):
        return self.pipeline_prompt.format(
            chef_name=chef_name,
            example_dish_name=example_dish_name,
            user_dish_name=user_dish_name
        )

# Usage
chatbot = CookingShowChatbot()
scenario = chatbot.run_scenario(chef_name="Gordon Ramsay", example_dish_name="Beef Wellington", user_dish_name="Vegan Lasagna")
print(scenario)

Welcome to the cooking show! Today, you're channeling the spirit of Chef Gordon Ramsay.
        Remember when Chef Gordon Ramsay made that delicious Beef Wellington? It was a hit!
        
        Now, it's your turn! Show us how you make your Vegan Lasagna. Let's get cooking!


In [ ]:
print(llm(scenario))



Step 1: Preheat oven to 375°F.

Step 2: Begin by preparing the vegan ricotta cheese. In a food processor, combine 2 containers of extra firm tofu, 2 tablespoons of olive oil, 2 cloves of garlic, 2 tablespoons of nutritional yeast, 1 teaspoon of salt, 1 teaspoon of dried oregano, and 1 teaspoon of dried basil. Blend until creamy and set aside.

Step 3: In a large saucepan, heat 2 tablespoons of olive oil over medium-high heat and add 1 diced onion. Cook for 3 minutes, stirring occasionally.

Step 4: Add 3 cloves of minced garlic and cook for 1 minute.

Step 5: Add 2 cans of crushed tomatoes and 1 teaspoon of salt. Bring to a simmer and cook for 10 minutes, stirring occasionally.

Step 6: In a 9x13 inch baking dish, spread 1 cup of the tomato sauce evenly over the bottom.

Step 7: Layer 4 uncooked lasagna noodles on top of the sauce.

Step 8: Spread half of the vegan ricotta cheese over the noodles, followed by 1 cup of the tomato sauce.

Step 9: Add 4 more noodles and spread the remai